In [1]:
import numpy as np
import numpy.random as npr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format = 'retina'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from src.baselines import CNNModel, CARPModel, MSModel, LinearModel
from src.torch_helpers import NamedTensorDataset
from src.datamodule import PeptideDataModule
from pytorch_lightning import Trainer
from src.torch_helpers import NoValProgressBar
from src.constants import MSConstants
C = MSConstants()

torch.manual_seed(0);

/home/gridsan/mmurphy/.conda/envs/MSPretraining/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# from src.torch_helpers import start_tensorboard
# start_tensorboard(login_node='login-2')

In [5]:
# generate negatives by producing tons of shuffled sequences
# then cluster, and only take clusters containing sufficiently many positives

# from src.cdhit import CDHIT

# def generate_negatives(sequences, num_shuffles=10, min_frac=0.1, random_state=0):
#     pos_seqs = list(sequences)
#     neg_seqs = []
#     rng = npr.RandomState(random_state)
#     for n in range(num_shuffles):
#         neg_seqs += [''.join(rng.permutation(list(s))) for s in pos_seqs]

#     seqs = np.array(pos_seqs + neg_seqs)
#     ids = np.array([1]*len(pos_seqs) + [0]*len(neg_seqs))
#     clusters = np.array(CDHIT(threshold=0.5,word_length=3).fit_predict(seqs))
#     pos_frac = pd.DataFrame([clusters,ids],index=['clusters','ids']).T.groupby('clusters').mean()['ids']
#     pos_clusters = set(pos_frac[pos_frac>min_frac].index)
    
#     pos_seqs = set(pos_seqs)
#     negatives = [s for i,s,c in zip(ids,seqs,clusters) if i==0 and c in pos_clusters and s not in pos_seqs]
    
#     rng.shuffle(negatives)
#     negatives = negatives[:len(sequences)]
    
#     assert len(negatives) == len(sequences)
    
#     return negatives

# Datasets

In [6]:
datasets = {}

### Mitochondrial targeting

In [7]:
df = pd.read_csv('./data/mitochondria_targeting.csv')
df = df[['Sequence','Mitochondrial Targeting Signal']].drop_duplicates(keep='first')
df = df.loc[df['Sequence'].map(lambda s: all([c==c.upper() and c in C.alphabet for c in s]))]
df = df.sample(frac=1.,random_state=0)

dataset = NamedTensorDataset(
    'mito',
    sequence=df['Sequence'],
    x=df['Sequence'].map(lambda s: [C.alphabet.index(c) for c in s]),
    x_mask=df['Sequence'].map(lambda s: [1]*len(s)),
    y=df['Mitochondrial Targeting Signal'].astype(int)
)

datasets[dataset.name] = dataset

### Cdc28 binding

In [8]:
df = pd.read_csv('./data/cdc28_binding.csv')
df = df[['Sequence','Cdc28 Binding']].drop_duplicates(keep='first')
df = df.loc[df['Sequence'].map(lambda s: all([c==c.upper() and c in C.alphabet for c in s]))]
df = df.sample(frac=1.,random_state=0)

dataset = NamedTensorDataset(
    'cdc28',
    sequence=df['Sequence'],
    x=df['Sequence'].map(lambda s: [C.alphabet.index(c) for c in s]),
    x_mask=df['Sequence'].map(lambda s: [1]*len(s)),
    y=df['Cdc28 Binding'].astype(int)
)

datasets[dataset.name] = dataset

len(dataset)

5348

### Signal peptide

In [9]:
with open('./data/train_set.fasta','r') as f:
    fasta = [l.strip() for l in f]
    df = pd.Series(fasta[::3]).str.extract(
        '>(?P<uniprot>[^\|]+)\|(?P<kingdom>[^|]+)\|(?P<type>[^|]+)\|(?P<partition>[^|]+)'
    )
    df['sequence'] = fasta[1::3]
    df['annotation'] = fasta[2::3]
df = df.loc[df['sequence'].map(lambda s: all([c==c.upper() and c in C.alphabet for c in s]))]
df = df.sample(frac=1.,random_state=0)

dataset = NamedTensorDataset(
    'signalp',
    sequence=df['sequence'],
    x=df['sequence'].map(lambda s: [C.alphabet.index(c) for c in s]),
    x_mask=df['sequence'].map(lambda s: [1]*len(s)),
    y=(df['type']!='NO_SP').astype(int)
)

datasets[dataset.name] = dataset

len(dataset)

20290

### HLA binding

In [10]:
pos_seqs = !cat ./data/mlci2012/binding_*.txt
neg_seqs = !cat ./data/mlci2012/nonbinding_*.txt
df = pd.DataFrame({
    'sequence': pos_seqs + neg_seqs,
    'hla_binding': [1]*len(pos_seqs) + [0]*len(neg_seqs)
})
df = df.drop_duplicates(subset=['sequence'])
df = df.loc[df['sequence'].map(lambda s: all([c==c.upper() and c in C.alphabet for c in s]))]
df = df.sample(frac=1.,random_state=0)

dataset = NamedTensorDataset(
    'hla',
    sequence=df['sequence'],
    x=df['sequence'].map(lambda s: [C.alphabet.index(c) for c in s]),
    x_mask=df['sequence'].map(lambda s: [1]*len(s)),
    y=df['hla_binding'].astype(int)
)

datasets[dataset.name] = dataset

len(dataset)

16665

In [11]:
from torch.utils.data import Subset
from collections import Counter

npr.seed(0)

MIN_LENGTH = 5
MAX_LENGTH = 100

for d in datasets:
    ctr = Counter()
    idxs = [
        i 
        for i,item in enumerate(datasets[d]) 
        if (len(item['sequence'])>=MIN_LENGTH) and (len(item['sequence'])<=MAX_LENGTH)
    ]
    npr.shuffle(idxs)
    datasets[d] = Subset(datasets[d], idxs)
    ctr = Counter([item['y'] for item in datasets[d]])
    print(d, ctr)

mito Counter({0: 3960, 1: 161})
cdc28 Counter({0: 4040, 1: 80})
signalp Counter({0: 15625, 1: 4665})
hla Counter({1: 9720, 0: 6945})


# Models

In [12]:
models = {}

In [13]:
MODEL_DIM = 128
LR = 5e-4

### MS pretraining

In [14]:
[last_ckpt] = !ls -t1 ./version_17869980/checkpoints/*.ckpt | head -n1
last_ckpt

'./version_17869980/checkpoints/epoch=92-step=114948-best.ckpt'

In [15]:
model = lambda output_dim: MSModel(
    checkpoint = last_ckpt,
    model_dim = MODEL_DIM,
    output_dim = output_dim,
    fixed_weights = True,
    naive = False,
    lr = LR
)
models['ms_pretrained_frozen'] = model

In [16]:
model = lambda output_dim: MSModel(
    checkpoint = last_ckpt,
    model_dim = MODEL_DIM,
    output_dim = output_dim,
    fixed_weights = False,
    naive = False,
    lr = LR
)
models['ms_pretrained_finetune'] = model

### Linear baseline

In [17]:
model = lambda output_dim: LinearModel(
    output_dim = output_dim,
    model_dim = MODEL_DIM,
    num_residues = len(C.alphabet),
    lr = LR,
)
models['linear'] = model

### CNN baseline

In [18]:
# https://arxiv.org/pdf/2011.03443.pdf
# differences: 128 vs 1024, attention vs max pool
model = lambda output_dim : CNNModel(
    output_dim = output_dim,
    model_dim = MODEL_DIM,
    model_depth = 3,
    kernel_size = 5,
    num_residues = len(C.alphabet),
    dropout = 0.,
    lr = LR,
)
models['cnn'] = model

### Random

In [19]:
model = lambda output_dim : MSModel(
    checkpoint = last_ckpt,
    model_dim = MODEL_DIM,
    output_dim = output_dim,
    fixed_weights = False,
    naive = True,
    lr = LR,
)
models['random_frozen'] = model

In [20]:
model = lambda output_dim : MSModel(
    checkpoint = last_ckpt,
    model_dim = MODEL_DIM,
    output_dim = output_dim,
    fixed_weights = True,
    naive = True,
    lr = LR
)
models['random_finetune'] = model

### Large language model

In [21]:
model = lambda output_dim : CARPModel(
    output_dim = output_dim,
    fixed_weights = True,
    lr = LR,
)
models['carp_pretrained_frozen'] = model

In [22]:
model = lambda output_dim : CARPModel(
    output_dim = output_dim,
    fixed_weights = False,
    lr = LR,
)
models['carp_pretrained_finetune'] = model

In [23]:
for MODEL in models.keys():
    for DATASET in datasets.keys():
        name = MODEL+'_'+DATASET
        !rm -rf ./lightning_logs/$name
!rm -rf ./lightning_logs/version_$SLURM_JOBID

In [24]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.utils.data import DataLoader
from src.torch_helpers import zero_padding_collate
from src.cdhit import cdhit_split

seed_everything(0, workers=True)

aucs = {}

for MODEL in models.keys():
    for DATASET in datasets.keys():
        name = MODEL+'_'+DATASET
        !rm -rf ./lightning_logs/$name
!rm -rf ./lightning_logs/version_$SLURM_JOBID

for DATASET in datasets.keys():
    dataset = datasets[DATASET]
    
    sequences = [item['sequence'] for item in dataset]
    train_val_seqs, test_seqs, train_val_dataset, test_dataset = cdhit_split(
        sequences,
        dataset,
        split=2./3,
        threshold=0.5,
        word_length=3
    )
    
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=len(test_dataset),
        collate_fn=zero_padding_collate,
        num_workers=1,
        shuffle=False,
        drop_last=False
    )
    
    dm = PeptideDataModule(
        train_val_dataset,
        batch_size=256,
        val_batch_size=-1,
        train_val_split=0.5,
        cdhit_threshold=0.5,
        cdhit_word_length=3,
        num_workers=4
    )
    dm.setup()
    
    OUTPUT_DIM = 1

    for MODEL in models.keys():
        name = MODEL+'_'+DATASET
        
        torch.manual_seed(0)
        
        model = models[MODEL](OUTPUT_DIM)
        
        model.name = DATASET
        
        trainer = Trainer(
            gpus=1,
            precision=32,
            max_epochs=1000,
            min_epochs=30,
            callbacks=[
                NoValProgressBar(),
                EarlyStopping(
                    monitor=f'val_auc_{DATASET}',
                    mode='max',
                    patience=10
                ),
                ModelCheckpoint(
                    monitor=f'val_auc_{DATASET}', 
                    mode='max',
                    save_top_k=1,
                    filename='{epoch}-{step}-best'
                )
            ]
        )

        trainer.fit(model, dm)
        
        [best_ckpt] = !ls -t1 ./lightning_logs/version_$SLURM_JOBID/checkpoints/*.ckpt | head -n1
        checkpoint = torch.load(best_ckpt)
        model.load_state_dict(checkpoint['state_dict'])
        
        metrics, = trainer.test(model, test_dataloader)
        
        aucs[(DATASET,MODEL)] = metrics[f'test_auc_{DATASET}']
        
        print(DATASET, MODEL)

        !mv ./lightning_logs/version_$SLURM_JOBID ./lightning_logs/latest
        !mv ./lightning_logs/latest ./lightning_logs/$name

Global seed set to 0
Multiprocessing is handled by SLURM.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-433d5424-0d96-1e9d-aaab-7d900550b968]

  | Name       | Type           | Params
----------------------------------------------
0 | encoder    | ByteNet        | 603 K 
1 | classifier | ESMAttention1d | 16.8 K
----------------------------------------------
16.8 K    Trainable params
603 K     Non-trainable params
620 K     Total params
2.481     Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


/home/gridsan/mmurphy/.conda/envs/MSPretraining/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1938: PossibleUserWarning: The number of training samples (5) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 55: 100%|██████████| 6/6 [00:50<00:00,  8.42s/it, loss=0.0515, v_num=1.79e+7]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-433d5424-0d96-1e9d-aaab-7d900550b968]
/home/gridsan/mmurphy/.conda/envs/MSPretraining/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.53it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_auc_mito         0.8738208413124084
        test_loss           0.11530361324548721
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
mito ms_pretrained_frozen


...

Epoch 28: 100%|██████████| 21/21 [00:58<00:00,  2.79s/it, loss=0.02, v_num=1.79e+7]  

Trainer was signaled to stop but required minimum epochs (30) or minimum steps (None) has not been met. Training will continue...


Epoch 29: 100%|██████████| 21/21 [01:00<00:00,  2.88s/it, loss=0.00801, v_num=1.79e+7]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-433d5424-0d96-1e9d-aaab-7d900550b968]


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.32it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_auc_hla          0.7286828756332397
        test_loss            0.67215496301651
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
hla carp_pretrained_finetune


In [29]:
df = pd.DataFrame(aucs,index=[0])
df.columns.names = ['dataset','model']
df = df.T
df = df.pivot_table(index='dataset',columns='model')[0]
# df['ms_naive'] = [aucs[('cdc28','ms_naive')],aucs[('mito','ms_naive')],aucs[('signalp','ms_naive')]]
df.round(2).T[['mito','cdc28','signalp','hla']]

dataset,mito,cdc28,signalp,hla
model,,,,
carp_pretrained_finetune,0.84,0.80,0.99,0.73
carp_pretrained_frozen,0.85,0.79,0.99,0.72
cnn,0.79,0.68,0.93,0.75
linear,0.83,0.64,0.84,0.66
ms_pretrained_finetune,0.86,0.76,0.99,0.72
ms_pretrained_frozen,0.87,0.80,0.97,0.70
random_finetune,0.85,0.75,0.95,0.68
random_frozen,0.85,0.77,0.98,0.73


In [26]:
# from sklearn.metrics import confusion_matrix

# model = model.cpu()
# model.eval()

# ys = []
# y_preds = []

# for batch in dm.val_dataloader():
#     y_pred = model.predict_step(batch, 0).detach().cpu().numpy()
#     y = batch['y'].cpu().numpy()
#     ys.append(y)
#     y_preds.append(y_pred)
# y = np.concatenate(ys)
# y_pred = np.concatenate(y_preds)

# for k in range(y.shape[1]):
#     plt.figure(figsize=(4,4))
#     sns.heatmap(
#         confusion_matrix(y[:,k], y_pred[:,k]>0.5),
#         annot=True, fmt='d', cmap='Blues'
#     )